Pairs preparation and the VGG16 with Siamese model (with the VGG16 imported from tensorflow.keras.applications)

In [ ]:
#importing the needed libraries
import numpy as np
from PIL import Image
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.layers import Input, Dense, Dropout, Convolution2D, MaxPooling2D, Flatten, BatchNormalization, Conv2D, MaxPool2D, Activation
from keras.models import Sequential, Model
from keras.losses import binary_crossentropy
from keras.optimizers import Adam
from keras.metrics import Recall, Precision
from tensorflow.keras.applications import VGG16

In [ ]:
def read_jpeg_image(filename):
    # Open the image file
    img = Image.open(filename)

    # Convert the image to a NumPy array
    img_array = np.array(img)

    return img_array

In [ ]:
#the path of the folder containing the genuine and fake folders of each person
path=path_of_folder

To read genuine images

In [ ]:
import os
n = number_of_persons
folder_paths = [f"path"+"/person{i}/original" for i in range(0, n)]
original_files = []

for folder_path in folder_paths:
    original_files.append(os.listdir(folder_path))
    print(folder_path)

for file in original_files:
    print(file)


To read fakes images

In [ ]:
import os
n = number_of_persons
folder_paths = [f"path"+"/person{i}/Deepfakes_1" for i in range(0, n)]
deepfakes1_files = []

for folder_path in folder_paths:
    deepfakes1_files.append(os.listdir(folder_path))
    print(folder_path)
for file in deepfakes1_files:
    print(file)


In [ ]:
import os
n = number_of_persons
folder_paths = [f"path"+"/person{i}/Deepfakes_2" for i in range(0,n)]
deepfakes2_files = []

for folder_path in folder_paths:
    deepfakes2_files.append(os.listdir(folder_path))
    print(folder_path)
for file in deepfakes2_files:
    print(file)


In [ ]:
import os
n = number_of_persons
folder_paths = [f"path"+"/person{i}/Face2Face_1" for i in range(0, n)]
face2face1_files = []

for folder_path in folder_paths:
    face2face1_files.append(os.listdir(folder_path))
    print(folder_path)
for file in face2face1_files:
    print(file)

In [ ]:
import os
n = number_of_persons
folder_paths = [f"path"+"/person{i}/Face2Face_2" for i in range(0, n)]
face2face2_files = []

for folder_path in folder_paths:
    face2face2_files.append(os.listdir(folder_path))
    print(folder_path)
for file in face2face2_files:
    print(file)


To make the pairs:

Genuine pairs consist of two genuine images of a person. Fake pairs consist of two images of a person: one genuine and one fake.

In [ ]:
#number_of_pairs = number_of_genuine_pairs = number_of_fake_pairs
total_sample_size= number_of_pairs
path = path_of_folder
np = number_of_images
n = number_of_persons
def get_data( total_sample_size):
    #read the image
    image = read_jpeg_image(filename='path'+'/person0/original/1.jpg')
    #get the size
    dim1 = image.shape[0]
    dim2 = image.shape[1]
    dim3 = image.shape[2]

    genuine_count=0

    #to initialize the numpy array with the shape of [total_sample_size, 2, dim1, dim2]
    x_geuine_pair = np.zeros([total_sample_size, 2, image.shape[0] ,image.shape[1],3])  # 2 for pairs
    y_genuine = np.zeros([total_sample_size, 1])
    fake_count=0
    x_fake_pair = np.zeros([total_sample_size, 2, image.shape[0] ,image.shape[1],3])  # 2 for pairs
    y_fake = np.zeros([total_sample_size, 1])
#for first set of genuine pairs
    for i in range (n):
        for j in range (np):
          img1 = read_jpeg_image('path'+'/person' +str(i)+'/original/'+ original_files[i][j])
          img2 = read_jpeg_image('path'+'/person' +str(i)+'/original/'+ original_files[i][j])
          x_geuine_pair[genuine_count, 0 ] = img1
          x_geuine_pair[genuine_count, 1] = img2
          y_genuine[genuine_count] = 1
          genuine_count+=1
    #for first set of fake pairs (using genuine and Deepfakes_1)
    for i in range (n):
        for j in range (np):
          img1 = read_jpeg_image('path'+'/person' +str(i)+'/original/'+ original_files[i][j])
          img2 = read_jpeg_image('path'+'/person' +str(i)+'/Deepfakes_1/'+ deepfakes1_files[i][j])
          x_fake_pair[fake_count, 0 ] = img1
          x_fake_pair[fake_count, 1] = img2
          y_fake[fake_count] = 0
          fake_count+=1

      #for second set of genuine pairs
    for i in range (n):
        for j in range (np-1):
          img1 = read_jpeg_image('path'+'/person' +str(i)+'/original/'+ original_files[i][j])
          img2 = read_jpeg_image('path'+'/person' +str(i)+'/original/'+ original_files[i][j+1])
          x_geuine_pair[genuine_count, 0 ] = img1
          x_geuine_pair[genuine_count, 1] = img2
          y_genuine[genuine_count] = 1
          genuine_count+=1
    #for second set of fake pairs (using genuine and Deepfakes_2)
    for i in range (n):
        for j in range (np-1):
          img1 = read_jpeg_image('path'+'/person' +str(i)+'/original/'+ original_files[i][j])
          img2 = read_jpeg_image('path'+'/person' +str(i)+'/Deepfakes_2/'+ deepfakes2_files[i][j])
          x_fake_pair[fake_count, 0 ] = img1
          x_fake_pair[fake_count, 1] = img2
          y_fake[fake_count] = 0
          fake_count+=1
    #for third set of genuine pairs
    for i in range (n):
        for j in range (np-3):
          img1 = read_jpeg_image('path'+'/person' +str(i)+'/original/'+ original_files[i][j])
          img2 = read_jpeg_image('path'+'/person' +str(i)+'/original/'+ original_files[i][j+3])
          x_geuine_pair[genuine_count, 0 ] = img1
          x_geuine_pair[genuine_count, 1] = img2
          y_genuine[genuine_count] = 1
          genuine_count+=1
    #for third set of fake pairs (using genuine and Face2Face_1)
    for i in range (n):
        for j in range (np-3):
          img1 = read_jpeg_image('path'+'/person' +str(i)+'/original/'+ original_files[i][j])
          img2 = read_jpeg_image('path'+'/person' +str(i)+'/Face2Face_1/'+ face2face1_files[i][j])
          x_fake_pair[fake_count, 0 ] = img1
          x_fake_pair[fake_count, 1] = img2
          y_fake[fake_count] = 0
          fake_count+=1

      #for fourth set of genuine pairs
    for i in range (n):
        for j in range (np-2):
          img1 = read_jpeg_image('path'+'/person' +str(i)+'/original/'+ original_files[i][j])
          img2 = read_jpeg_image('path'+'/person' +str(i)+'/original/'+ original_files[i][j+2])
          x_geuine_pair[genuine_count, 0 ] = img1
          x_geuine_pair[genuine_count, 1] = img2
          y_genuine[genuine_count] = 1
          genuine_count+=1
    #for fourth set of fake pairs (using genuine and Face2Face_2)
    for i in range (n):
        for j in range (np-2):
          img1 = read_jpeg_image('path'+'/person' +str(i)+'/original/'+ original_files[i][j])
          img2 = read_jpeg_image('path'+'/person' +str(i)+'/Face2Face_2/'+ face2face2_files[i][j])
          x_fake_pair[fake_count, 0 ] = img1
          x_fake_pair[fake_count, 1] = img2
          y_fake[fake_count] = 0
          fake_count+=1

    #to concatenate, genuine pairs and fake pairs to get the whole data X and Y
    X = np.concatenate([x_geuine_pair, x_fake_pair], axis=0)/255
    Y = np.concatenate([y_genuine, y_fake], axis=0)
    print(genuine_count)
    print(fake_count)
    return X, Y

In [ ]:
#to get X and Y
X, Y = get_data(total_sample_size)

In [ ]:
# to get the training and testing data
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=.3)

The VGG16 model using "from tensorflow.keras.applications import VGG16"

In [ ]:
def build_vgg16_base(input_shape):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

    base = base_model.output
    flat = Flatten()(base)
    x = Dense(4096, activation='relu')(flat)
    x = Dense(4096, activation='relu')(x)
    dense = Dense(50, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=dense)

    return model


In [ ]:
#to get the input shape
input_shape = x_train.shape[2:]

In [ ]:
#base_network will be used in the siamese model
base_network = build_vgg16_base(input_shape=input_shape)

In [ ]:
#the siamese model
from keras.layers import concatenate
img_a =  Input(shape=input_shape)
img_b =  Input(shape=input_shape)
img_a_feat = base_network(img_a)
img_b_feat = base_network(img_b)
#combined features
combined_features = concatenate([img_a_feat, img_b_feat], name = 'merge_features')
#dense layers, batchNormalization and sigmoid
combined_features = Dense(100, activation = 'relu')(combined_features)
combined_features = Activation('relu')(combined_features)
combined_features = Dense(20, activation = 'relu')(combined_features)
combined_features = BatchNormalization()(combined_features)
combined_features = Dense(1, activation = 'sigmoid')(combined_features)
similarity_model = Model(inputs = [img_a, img_b], outputs = [combined_features], name = 'Similarity_Model')
similarity_model.summary()

In [ ]:
#to compile the siamese model
similarity_model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy',tf.metrics.Recall(), tf.metrics.Precision()])

In [ ]:
#the inputs for the siamese model
img_1 = x_train[:, 0]
img_2 = x_train[:, 1]

In [ ]:
#to train the siamese model
loss_history =  similarity_model.fit([img_1, img_2], y_train,
          batch_size=10, verbose=2, epochs=30)

In [ ]:
#to evaluate the siamese model
pred = similarity_model.evaluate([x_test[:, 0], x_test[:, 1]], y_test)

In [ ]:
#for siamese model prediction
pred = similarity_model.predict([x_test[:, 0], x_test[:, 1]])

In [ ]:
pred

In [ ]:
y_test

In [ ]:
#importing the needed library
from sklearn .metrics import roc_auc_score
auc = np.round(roc_auc_score(y_test, pred), 3)
print("Auc for our sample data is {}".format(auc))

In [ ]:
#importing the needed libraries
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

In [ ]:
# to calculate the ROC curve
fpr, tpr, thresholds = roc_curve(y_test, pred)
roc_auc = auc(fpr, tpr)
# to plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()